# Texas Unemployment and Wages Review and Analytics 

## Import the required libraries and dependencies

In [49]:
import plotly.express as px
import pandas as pd
import os
import numpy as np
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv

### Prep the Mapbox API Access Token

In [50]:
# Read the Mapbox API access token from the .env file
load_dotenv()
mapbox_api_access_token = os.getenv("MAPBOX_API_ACCESS_TOKEN")

# Confirm that the mapbox_api_access_token is available
if not mapbox_api_access_token:
    print("Error with the Mapbox API access token. Check the .env file.")

In [51]:
# Set the Mapbox API access token
px.set_mapbox_access_token(mapbox_api_access_token)

### Prepare the DataFrame

In [52]:
# Read in the All_Sector_Employment_and_Wages_Report_by_area.csv and Finance_and_Insurance_Sector_Employment_and_Wages_Report_by_area.csv files into a DataFrame
employment_wages_df = pd.read_csv(
    Path("../Resources/All_Sector_Employment_and_Wages_Report_by_area.csv"),
    index_col="Year",
    infer_datetime_format=True,
).drop_duplicates()

finance_employment_wages_df = pd.read_csv(
    Path("../Resources/Finance_and_Insurance_Sector_Employment_and_Wages_Report_by_area.csv"),
    index_col="Year",
    infer_datetime_format=True,
).drop_duplicates()

us_Inflation_df = pd.read_csv(
    Path("../Resources/US_Inflation.csv"),
    index_col="Year",
    infer_datetime_format=True,    
).drop_duplicates()

tx_unemployment_rate_df = pd.read_csv(
    Path("../Resources/Texas_Unemployment_Rate_by_area.csv"),
    index_col="Year",
    infer_datetime_format=True,
).drop_duplicates()

tx_employment_wages_df = pd.read_csv(
    Path("../Resources/Texas_Employmnet_and_Wages.csv"),
    index_col="Year",
    infer_datetime_format=True,
).drop_duplicates()

#Review of Dataframes needed for questions
display(us_Inflation_df.head())
display(tx_unemployment_rate_df.head())


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
2006,0.040,0.036,0.034,0.036,0.040,0.042,0.041,0.039,0.020,0.014,0.020,0.025
2007,0.021,0.024,0.028,0.026,0.027,0.027,0.023,0.019,0.028,0.036,0.044,0.041
2008,0.043,0.041,0.040,0.039,0.041,0.049,0.055,0.053,0.050,0.037,0.011,0.000
2009,-0.001,0.000,-0.004,-0.006,-0.010,-0.012,-0.020,-0.015,-0.014,-0.002,0.019,0.028
2010,0.026,0.022,0.023,0.022,0.020,0.011,0.013,0.012,0.011,0.012,0.011,0.014


,Period,Area,Area Type,Adjustment,Employment,Civilian Labor Force,Unemployment,Unemployment_Rate,Unnamed: 9,Unnamed: 10
Year,,,,,,,,,,
2006,Annual,Texas,Texas,Not Adjusted,"10,770,786","11,335,112","564,326",0.050,NaN,NaN
2006,January,Texas,Texas,Not Adjusted,"10,575,831","11,169,955","594,124",0.053,NaN,NaN
2006,January,Texas,Texas,Adjusted,"10,647,444","11,240,453","593,009",0.053,NaN,NaN
2006,February,Texas,Texas,Not Adjusted,"10,620,818","11,224,932","604,114",0.054,NaN,NaN
2006,February,Texas,Texas,Adjusted,"10,664,814","11,253,367","588,553",0.052,NaN,NaN


Pulling Applicable Data frames for Question 1: Correlation Between Inflation Vs Unemployment Rate

In [53]:
#Original Data frame pull and review
us_Inflation_df.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
2006,0.040,0.036,0.034,0.036,0.040,0.042,0.041,0.039,0.020,0.014,0.020,0.025
2007,0.021,0.024,0.028,0.026,0.027,0.027,0.023,0.019,0.028,0.036,0.044,0.041
2008,0.043,0.041,0.040,0.039,0.041,0.049,0.055,0.053,0.050,0.037,0.011,0.000
2009,-0.001,0.000,-0.004,-0.006,-0.010,-0.012,-0.020,-0.015,-0.014,-0.002,0.019,0.028
2010,0.026,0.022,0.023,0.022,0.020,0.011,0.013,0.012,0.011,0.012,0.011,0.014


In [80]:
#Verify information and dataType of Dataframe 
us_Inflation_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 2006 to 2021
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Jan     16 non-null     float64
 1   Feb     16 non-null     float64
 2   Mar     16 non-null     float64
 3   Apr     16 non-null     float64
 4   May     16 non-null     float64
 5   Jun     16 non-null     float64
 6   Jul     15 non-null     float64
 7   Aug     15 non-null     float64
 8   Sep     15 non-null     float64
 9   Oct     15 non-null     float64
 10  Nov     15 non-null     float64
 11  Dec     15 non-null     float64
dtypes: float64(12)
memory usage: 1.6 KB


In [78]:
#Create new Dataframe to show averages of inflation over the years based on the US Inflation Statistics
avg_us_inflation_df = us_Inflation_df.mean(axis=1)
avg_us_inflation_df.head()

Year
2006    0.032250
2007    0.028667
2008    0.038250
2009   -0.003083
2010    0.016417
dtype: float64

In [102]:
# Plotting the inflation rate to verify the information
avg_us_inflation_df.hvplot.line(
    title = "Average Inflation Rate 2006 - Present",
    xlabel= "Year", 
    ylabel= "Inflation Rate",
    color = "green"
)



:Curve   [Year]   (0)

Review of Unemployment Rate in Texas - Pulling and Scrubbing data for Texas

In [56]:
#Review of Dataframes needed for questions
display(tx_unemployment_rate_df.head())

,Period,Area,Area Type,Adjustment,Employment,Civilian Labor Force,Unemployment,Unemployment_Rate,Unnamed: 9,Unnamed: 10
Year,,,,,,,,,,
2006,Annual,Texas,Texas,Not Adjusted,"10,770,786","11,335,112","564,326",0.050,NaN,NaN
2006,January,Texas,Texas,Not Adjusted,"10,575,831","11,169,955","594,124",0.053,NaN,NaN
2006,January,Texas,Texas,Adjusted,"10,647,444","11,240,453","593,009",0.053,NaN,NaN
2006,February,Texas,Texas,Not Adjusted,"10,620,818","11,224,932","604,114",0.054,NaN,NaN
2006,February,Texas,Texas,Adjusted,"10,664,814","11,253,367","588,553",0.052,NaN,NaN


In [81]:
#Verify Data types before running data analytics 
tx_unemployment_rate_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57050 entries, 2006 to 2021
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Period                57050 non-null  object 
 1   Area                  57050 non-null  object 
 2   Area Type             57050 non-null  object 
 3   Adjustment            57050 non-null  object 
 4   Employment            57050 non-null  object 
 5   Civilian Labor Force  57050 non-null  object 
 6   Unemployment          57050 non-null  object 
 7    Unemployment_Rate    57050 non-null  float64
 8   Unnamed: 9            0 non-null      float64
 9   Unnamed: 10           0 non-null      float64
dtypes: float64(3), object(7)
memory usage: 4.8+ MB


In [44]:
#Pulling just the Annual Period for Unemployment Rate basis of information 
filtered_tx_annual_unemployment_df = tx_unemployment_rate_df[(tx_unemployment_rate_df['Period'] =="Annual") & (tx_unemployment_rate_df['Area'] =="Texas")] 
filtered_tx_annual_unemployment_df



,Period,Area,Area Type,Adjustment,Employment,Civilian Labor Force,Unemployment,Unemployment Rate,Unnamed: 9,Unnamed: 10
Year,,,,,,,,,,
2006,Annual,Texas,Texas,Not Adjusted,"10,770,786","11,335,112","564,326",0.050,NaN,NaN
2007,Annual,Texas,Texas,Not Adjusted,"10,940,176","11,441,054","500,878",0.044,NaN,NaN
2008,Annual,Texas,Texas,Not Adjusted,"11,110,499","11,677,743","567,244",0.049,NaN,NaN
2009,Annual,Texas,Texas,Not Adjusted,"11,043,167","11,944,138","900,971",0.075,NaN,NaN
2010,Annual,Texas,Texas,Not Adjusted,"11,255,444","12,260,100","1,004,656",0.082,NaN,NaN
2011,Annual,Texas,Texas,Not Adjusted,"11,498,869","12,499,595","1,000,726",0.080,NaN,NaN
2012,Annual,Texas,Texas,Not Adjusted,"11,794,975","12,639,465","844,490",0.067,NaN,NaN
2013,Annual,Texas,Texas,Not Adjusted,"12,022,272","12,832,035","809,763",0.063,NaN,NaN
2014,Annual,Texas,Texas,Not Adjusted,"12,333,076","13,006,202","673,126",0.052,NaN,NaN


In [64]:
#Pulling Relevant Unemployment Rate for Texas, and dropping the other information
Scrubbed_annual_tx_unemployment_rate_df = filtered_tx_annual_unemployment_df.drop(['Period','Area','Area Type','Adjustment','Employment','Civilian Labor Force','Unemployment','Unnamed: 9','Unnamed: 10'],1)
Scrubbed_annual_tx_unemployment_rate_df



,Unemployment Rate
Year,
2006,0.050
2007,0.044
2008,0.049
2009,0.075
2010,0.082
2011,0.080
2012,0.067
2013,0.063
2014,0.052


In [67]:
Scrubbed_annual_tx_unemployment_rate_df.hvplot.bar(
    title = "Texas Unemployment Rate 2006 - Present",
    xlabel= "Year", 
    ylabel= "Unemployment Rate",
    color = "Orange"
) 


:Bars   [Year]   ( Unemployment Rate )

Verifying and Combing to compare the US Inflation Rate and comparing it against the Texas Unemployment Rate

In [94]:
#Reviewing and Comparing the US Inflation Rate, and comparing it to the Texas Unemployment - Concatenating the Dataframes

USInflation_Vs_TXUnemployment_df = pd.concat(
    [avg_us_inflation_df,Scrubbed_annual_tx_unemployment_rate_df],
    axis = 1,
    join = "inner",
 )
#Updating Column Names to reflect Inflation Rate and Unemployment Rate for graphing
USInflation_Vs_TXUnemployment_df.columns = ['Inflation_Rate','Unemployment_Rate']
USInflation_Vs_TXUnemployment_df


,Inflation_Rate,Unemployment_Rate
Year,,
2006,0.032250,0.050
2007,0.028667,0.044
2008,0.038250,0.049
2009,-0.003083,0.075
2010,0.016417,0.082
2011,0.031500,0.080
2012,0.020833,0.067
2013,0.014583,0.063
2014,0.016250,0.052


In [101]:
#Graphical analysis of the information US Inflation vs Tx Unemployment Line vs Bar

Scrubbed_annual_tx_unemployment_rate_df.hvplot.bar(
    label = "Tx Unemployment Rate",
    color = "Orange") * avg_us_inflation_df.hvplot.line(
        label = "US Average Inflation Rate",
        color = "Green") 

:Overlay
   .Bars.Tx_Unemployment_Rate       :Bars   [Year]   ( Unemployment Rate )
   .Curve.US_Average_Inflation_Rate :Curve   [Year]   (0)

In [106]:
#Graphical Analysis of the Dataframe combined together to compare Inflation and Tx Unemployment
USInflation_Vs_TXUnemployment_df.hvplot(
    title = "US Inflation vs. TX Unemployment from 2006 - 2020",
    xlabel= "Year",   
) 


:NdOverlay   [Variable]
   :Curve   [Year]   (value)